In [1]:
INPUT_NODES = 48
OUTPUT_NODES = 2

# Reference: https://discuss.pynq.io/t/tutorial-pynq-dma-part-2-using-the-dma-from-pynq/3134

In [2]:
from pynq import Overlay, allocate, PL
import os
import numpy as np
# from fxpmath import Fxp
import time

PL.reset()

cwd = os.getcwd()
bitstream_path = os.path.join(cwd, "fifo.bit")

# ol.free()
ol = Overlay(bitstream_path)

ol.ip_dict.keys()

dict_keys(['dma', 'mlp_0', 'ps'])

In [3]:
dma = ol.dma
dma_send = dma.sendchannel
dma_receive = dma.recvchannel

dma.sendchannel.stop()
dma.recvchannel.stop()
dma.sendchannel.start()
dma.recvchannel.start()

mlp = ol.mlp_0

CONTROL_REGISTER = 0x0
mlp.write(CONTROL_REGISTER, 0x81)

input_buffer = allocate(shape=(INPUT_NODES,), dtype=np.uint32)
output_buffer = allocate(shape=(OUTPUT_NODES,), dtype=np.uint32)

In [4]:
mlp.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [5]:
dma.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=0, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=0),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=0),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SG

In [6]:
import signal

def handler(signum, frame):
    raise Exception("Timeout")
    
signal.signal(signal.SIGALRM, handler)
signal.alarm(10)

def get_fpga_output(input_data):
    try:
        if len(input_data) != INPUT_NODES:
            print("Input data length doesn't match neural network input")
            return
        for i in range(len(input_data)):
            input_buffer[i] = input_data[i]
        # send input to FPGA and wait for output
        dma_send.transfer(input_buffer)
        print(f"Send Channel Running: {dma_send.running}, Idle: {dma_send.idle}")
        print(f"Receive Channel Running: {dma_receive.running}, Idle: {dma_receive.idle}")
        dma_send.wait()
        print(f"DMA Send Complete")
        dma_receive.transfer(output_buffer)
        print(f"Send Channel Running: {dma_send.running}, Idle: {dma_send.idle}")
        print(f"Receive Channel Running: {dma_receive.running}, Idle: {dma_receive.idle}")
        dma_receive.wait()
        print(f"DMA Receive Complete")
        print(f"Output buffer: {output_buffer}")
        action = np.argmax(output_buffer)
    except Exception as e:
        print("FPGA error")
        print(e)
        return
    return action
    

In [7]:
test_input = np.array([-0.136, -1.4479, -1.5651, -0.9784, -1.3872, -3.2972, 2.6525, 0.6658, 0.1994, 1.3327, 1.5996, 3.5092, 1.18812, -0.32259000000000004, -0.76517, 0.12972, 0.19205999999999995, -0.11801000000000003, 1.2500258275731746, 0.6610635498195313, 0.7215823432568178, 0.7926378817593819, 1.0179735047632625, 2.196188013103614, 1.724585081693565, 0.7355741465005414, 1.0517443639021795, 0.803182476153458, 1.0359329611514443, 2.1993563033305903, 0.08958212833281903, 0.2346326587274819, 0.3045064159112622, 1.0095246248703473, 0.3666000000000006, 1.1801000000000008, 12.452399999999999, 4.6165, 7.6517, 3.7775781288515713, 4.837956019908577, 10.466385327836475, 296.82227479999995, 42.912717779999994, 109.68472191999999, 34.69959259999999, 55.56940496, 251.50329057999997 ])
test_output = 1

# test_input = np.empty((INPUT_NODES, ), dtype=np.uint32)
# for i in range(INPUT_NODES):
#     test_input[i] = i

print(test_input)
# test_input = Fxp(test_input, signed=True, n_word=32, n_frac=15)

pred_output = get_fpga_output(test_input)

[-1.36000000e-01 -1.44790000e+00 -1.56510000e+00 -9.78400000e-01
 -1.38720000e+00 -3.29720000e+00  2.65250000e+00  6.65800000e-01
  1.99400000e-01  1.33270000e+00  1.59960000e+00  3.50920000e+00
  1.18812000e+00 -3.22590000e-01 -7.65170000e-01  1.29720000e-01
  1.92060000e-01 -1.18010000e-01  1.25002583e+00  6.61063550e-01
  7.21582343e-01  7.92637882e-01  1.01797350e+00  2.19618801e+00
  1.72458508e+00  7.35574147e-01  1.05174436e+00  8.03182476e-01
  1.03593296e+00  2.19935630e+00  8.95821283e-02  2.34632659e-01
  3.04506416e-01  1.00952462e+00  3.66600000e-01  1.18010000e+00
  1.24524000e+01  4.61650000e+00  7.65170000e+00  3.77757813e+00
  4.83795602e+00  1.04663853e+01  2.96822275e+02  4.29127178e+01
  1.09684722e+02  3.46995926e+01  5.55694050e+01  2.51503291e+02]
Send Channel Running: True, Idle: True
Receive Channel Running: True, Idle: False
DMA Send Complete
Send Channel Running: True, Idle: True
Receive Channel Running: True, Idle: False
FPGA error
Timeout


In [8]:
print(f"Output buffer: {output_buffer}")

Output buffer: [0 0]


In [9]:
del input_buffer, output_buffer